In [ ]:
import zipfile

zip_path = "/content/Books.csv.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
import pandas as pd

books = pd.read_csv("Books.csv", low_memory=False)

print(type(books))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [ ]:
books = books[['Book-Title']]
books = books.dropna()
books = books.head(10000)

print(books.shape)

(10000, 1)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features=3000)
vectors = tfidf.fit_transform(books['Book-Title'])

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=6, algorithm='brute')
model.fit(vectors)

NearestNeighbors(algorithm='brute', n_neighbors=6)

In [ ]:
def recommend(book_name):
    if book_name not in books['Book-Title'].values:
        print("❌ Book not found. Copy exact name.")
        return

    idx = books[books['Book-Title'] == book_name].index[0]
    distances, indices = model.kneighbors(vectors[idx])

    print("\n📚 Recommended books:")
    for i in indices[0][1:]:
        print("-", books.iloc[i]['Book-Title'])

In [ ]:
books['Book-Title'].head(20)

,Book-Title
0,Classical Mythology
1,Clara Callan
2,Decision in Normandy
3,Flu: The Story of the Great Influenza Pandemic...
4,The Mummies of Urumchi
5,The Kitchen God's Wife
6,What If?: The World's Foremost Military Histor...
7,PLEADING GUILTY
8,Under the Black Flag: The Romance and the Real...
9,Where You'll Find Me: And Other Stories


In [ ]:
#1
recommend("The Middle Stories")


📚 Recommended books:
- Slippage: Previously Poised, Previously Uncollected Stories
- Zlateh the Goat and Other Stories
- Bluebeard's Egg and Other Stories
- Circling the Drain : Stories
- Metamorphoses and Other Stories


In [ ]:
#2
def recommend():
    name = input("Enter book name: ").lower()

    # partial match
    matches = books[books['Book-Title'].str.lower().str.contains(name)]

    if len(matches) == 0:
        print("❌ Book not found")
        return

    # pick first matched book
    book_name = matches.iloc[0]['Book-Title']
    print(f"\n✅ Using book: {book_name}\n")

    index = books[books['Book-Title'] == book_name].index[0]
    distances, indices = model.kneighbors(vectors[index].reshape(1, -1))

    print("📚 Top 5 Recommended Books:\n")
    for i in indices[0][1:6]:
        print("•", books.iloc[i]['Book-Title'])

In [ ]:
recommend()

Enter book name: harry

✅ Using book: The Sorcerer's Companion: A Guide to the Magical World of Harry Potter

📚 Top 5 Recommended Books:

• Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
• Harry Potter and the Sorcerer's Stone (Book 1)
• Harry Potter and the Sorcerer's Stone (Book 1)
• Harry Potter and the Goblet of Fire (Book 4)
• Harry Potter and the Goblet of Fire (Book 4)


In [ ]:
#3
def book_app():
    print("📚 Book Recommendation System")
    print("Type 'exit' to stop\n")

    while True:
        name = input("Enter book name: ").strip()

        if name.lower() == "exit":
            print("👋 Thank you for using Book Recommender")
            break

        matches = books[books['Book-Title'].str.lower().str.contains(name.lower())]

        if len(matches) == 0:
            print("❌ Book not found. Try again.\n")
            continue

        book_name = matches.iloc[0]['Book-Title']
        print(f"\n✅ Using book: {book_name}\n")

        index = books[books['Book-Title'] == book_name].index[0]

        distances, indices = model.kneighbors(
            vectors[index].reshape(1, -1)
        )

        print("📖 Top 5 Recommended Books:")
        for i in indices[0][1:6]:
            print("•", books.iloc[i]['Book-Title'])
        print()

In [ ]:
book_app()

📚 Book Recommendation System
Type 'exit' to stop

Enter book name: harry

✅ Using book: The Sorcerer's Companion: A Guide to the Magical World of Harry Potter

📖 Top 5 Recommended Books:
• Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
• Harry Potter and the Sorcerer's Stone (Book 1)
• Harry Potter and the Sorcerer's Stone (Book 1)
• Harry Potter and the Goblet of Fire (Book 4)
• Harry Potter and the Goblet of Fire (Book 4)

Enter book name: reader

✅ Using book: Patty Jane's House of Curl (Ballantine Reader's Circle)

📖 Top 5 Recommended Books:
• Patty Jane's House of Curl
• Patty Janes House of Curl
• Evensong (Ballantine Reader's Circle)
• About Schmidt (Ballantine Reader's Circle)
• Pope Joan (Ballantine Reader's Circle)

Enter book name: jane doe

✅ Using book: Jane Doe

📖 Top 5 Recommended Books:
• Jane and Prudence
• Lauf, Jane, lauf. Roman.
• See Jane Run
• Jane Eyre
• Jane Eyre

Enter book name: exit
👋 Thank you for using Book Recommender
